In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import xgboost as xgb

import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.svm import SVC
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold, GridSearchCV, train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier

import optuna

In [2]:
!ls ../data/

movies                test.csv
sample_submission.csv train.csv


In [3]:
# cosnt variable

DATA_PATH = '../data/'
SEED = 222

In [4]:
train_data = pd.read_csv(f'{DATA_PATH}/train.csv')
test_data = pd.read_csv(f'{DATA_PATH}/test.csv')

In [5]:
train_data

,id,u,g,r,i,z,redshift,dered_u,dered_g,dered_r,dered_i,dered_z,nObserve,nDetect,airmass_u,airmass_g,airmass_r,airmass_i,airmass_z,class
0,0,23.263956,20.336773,19.009500,17.672439,16.939607,-0.000081,23.12426,20.25779,18.95512,17.63211,16.90894,18,18,1.189764,1.190681,1.188979,1.189355,1.190206,0
1,1,15.052147,14.061969,13.452418,13.268450,13.168941,0.004506,14.96642,14.00452,13.41139,13.23625,13.13468,1,1,1.253254,1.257836,1.248762,1.250997,1.255533,1
2,2,16.786385,15.825426,15.536318,15.393535,15.350032,0.000472,16.60765,15.68659,15.44004,15.32173,15.29608,2,2,1.022499,1.024105,1.020983,1.021730,1.023291,0
3,3,25.660638,21.188727,20.221158,19.894949,19.634649,0.000006,25.35365,20.99465,20.08727,19.79465,19.55518,4,3,1.205399,1.206058,1.204874,1.205120,1.205712,0
4,4,24.453429,20.699170,19.042368,18.324152,17.982649,-0.000033,23.77140,20.43384,18.86299,18.19028,17.87592,13,12,1.193946,1.194285,1.193738,1.193826,1.194099,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319995,319995,18.365648,17.240883,16.773898,16.484970,16.340403,0.051988,18.13280,17.04065,16.63744,16.38606,16.28741,2,2,1.093010,1.088828,1.097310,1.095145,1.090905,2
319996,319996,18.685639,17.665802,17.190434,16.816858,16.636688,0.087261,18.52074,17.51310,17.08711,16.74363,16.60173,2,2,1.051689,1.050838,1.052640,1.052152,1.051252,1
319997,319997,21.188972,18.423357,17.295610,16.901072,16.684127,-0.000010,20.98896,18.30760,17.21518,16.84030,16.63555,31,30,1.193259,1.194140,1.192509,1.192868,1.193683,0
319998,319998,18.847187,17.318605,16.505729,16.180571,15.911624,-0.000045,16.38998,15.40578,15.18229,15.19693,15.17935,1,1,1.172458,1.172561,1.172481,1.172454,1.172493,0


In [6]:
test_data

,id,u,g,r,i,z,redshift,dered_u,dered_g,dered_r,dered_i,dered_z,nObserve,nDetect,airmass_u,airmass_g,airmass_r,airmass_i,airmass_z
0,320000,19.143050,18.017200,17.473255,17.114411,16.926293,0.121303,19.07495,17.95495,17.43085,17.08381,16.90872,1,1,1.156348,1.151487,1.161321,1.158819,1.153902
1,320001,18.740910,17.170473,16.369084,15.986452,15.730708,-0.000067,15.07033,14.30976,14.39011,14.51600,14.63716,2,2,1.007637,1.007383,1.007975,1.007795,1.007500
2,320002,19.298048,17.880346,17.222135,16.809561,16.635239,0.110010,19.10036,17.77263,17.14601,16.74968,16.56888,1,1,1.380859,1.390490,1.371502,1.376145,1.385635
3,320003,18.077467,17.156943,16.904111,16.811800,16.795520,0.000030,17.97294,17.07353,16.84696,16.76995,16.76677,1,1,1.140796,1.140808,1.140903,1.140834,1.140787
4,320004,17.832467,16.764587,16.213526,15.872181,15.662084,0.068471,17.75447,16.67943,16.15628,15.83145,15.64335,2,2,1.111023,1.107658,1.114511,1.112750,1.109325
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79995,399995,19.673388,18.661662,18.270788,18.122501,18.062672,-0.000317,19.33614,18.39331,18.08600,17.98610,17.96699,17,17,1.185843,1.186322,1.185493,1.185652,1.186067
79996,399996,21.522869,20.728282,20.608695,21.004875,20.845799,0.065455,21.39935,20.63992,20.54506,20.95069,20.78268,8,4,1.284956,1.290772,1.279329,1.282120,1.287838
79997,399997,19.399065,18.241076,18.160492,18.197753,18.201831,-0.000842,18.91359,17.87533,17.90469,18.00426,18.04000,1,1,1.188475,1.188563,1.188518,1.188480,1.188503
79998,399998,18.899747,17.600868,17.132878,16.822347,16.594561,0.042817,18.77153,17.48798,17.05567,16.76595,16.55850,17,17,1.180429,1.181329,1.179654,1.180025,1.180862


In [7]:
# Separate X, y from train data
train_X = train_data.iloc[:, :-1]
train_y = train_data.iloc[:, -1:]['class']

In [8]:
train_X

,id,u,g,r,i,z,redshift,dered_u,dered_g,dered_r,dered_i,dered_z,nObserve,nDetect,airmass_u,airmass_g,airmass_r,airmass_i,airmass_z
0,0,23.263956,20.336773,19.009500,17.672439,16.939607,-0.000081,23.12426,20.25779,18.95512,17.63211,16.90894,18,18,1.189764,1.190681,1.188979,1.189355,1.190206
1,1,15.052147,14.061969,13.452418,13.268450,13.168941,0.004506,14.96642,14.00452,13.41139,13.23625,13.13468,1,1,1.253254,1.257836,1.248762,1.250997,1.255533
2,2,16.786385,15.825426,15.536318,15.393535,15.350032,0.000472,16.60765,15.68659,15.44004,15.32173,15.29608,2,2,1.022499,1.024105,1.020983,1.021730,1.023291
3,3,25.660638,21.188727,20.221158,19.894949,19.634649,0.000006,25.35365,20.99465,20.08727,19.79465,19.55518,4,3,1.205399,1.206058,1.204874,1.205120,1.205712
4,4,24.453429,20.699170,19.042368,18.324152,17.982649,-0.000033,23.77140,20.43384,18.86299,18.19028,17.87592,13,12,1.193946,1.194285,1.193738,1.193826,1.194099
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319995,319995,18.365648,17.240883,16.773898,16.484970,16.340403,0.051988,18.13280,17.04065,16.63744,16.38606,16.28741,2,2,1.093010,1.088828,1.097310,1.095145,1.090905
319996,319996,18.685639,17.665802,17.190434,16.816858,16.636688,0.087261,18.52074,17.51310,17.08711,16.74363,16.60173,2,2,1.051689,1.050838,1.052640,1.052152,1.051252
319997,319997,21.188972,18.423357,17.295610,16.901072,16.684127,-0.000010,20.98896,18.30760,17.21518,16.84030,16.63555,31,30,1.193259,1.194140,1.192509,1.192868,1.193683
319998,319998,18.847187,17.318605,16.505729,16.180571,15.911624,-0.000045,16.38998,15.40578,15.18229,15.19693,15.17935,1,1,1.172458,1.172561,1.172481,1.172454,1.172493


In [9]:
# LightGBM, Xgboost w. optuna

# LightGBM
lightgbm = lgb.LGBMClassifier(random_state=SEED)
# Xgboost
xgbboost = xgb.XGBClassifier(random_state=SEED)

def objective(trial):
    
    classifier = trial.suggest_categorical('classifier', ['KNeighbor', 'DecisionTree', 'SVM', 
                                                          'RandomForest', 'LightGBM', 'Xgboost'])
    
    # KFold
    kf = KFold(n_splits=5, random_state=SEED, shuffle=True)
    
    if classifier == 'KNeighbor':
        # KNN params
        knn_n_neighbors = trial.suggest_int('n_neighbors', 3, 10)
        knn_weights = trial.suggest_categorical('weights', ['uniform', 'distance'])
        knn_algorithm = trial.suggest_categorical('algorithm', ['auto', 'ball_tree', 'kd_tree', 'brute'])
        knn_leaf_size = trial.suggest_int('leaf_size', 10, 40)
        knn_p = trial.suggest_int('p', 1, 2)
        
        model = KNeighborsClassifier(
            n_neighbors=knn_n_neighbors,
            weights=knn_weights,
            algorithm=knn_algorithm,
            leaf_size=knn_leaf_size,
            p=knn_p
        )

    elif classifier == 'DecisionTree':
        # DecisionTree params
        dt_criterion = trial.suggest_categorical('criterion', ['gini', 'entropy'])
        dt_splitter = trial.suggest_categorical('splitter', ['best', 'random'])
        
        model = DecisionTreeClassifier(
            random_state=SEED,
            criterion=dt_criterion,
            splitter=dt_splitter
        )

    elif classifier == 'SVM':
        # SVM params
        svm_C = trial.suggest_categorical('svm_C', [0.1, 1, 10, 100, 1000])
        svm_degree = trial.suggest_categorical('svm_degree', [0, 1, 2, 3, 4, 5, 6])
        
        model = SVC(
            random_state=SEED,
            C=svm_C,
            degree=svm_degree
        )

    elif classifier == 'RandomForest':
        # RandomForest params
        rf_max_depth = trial.suggest_categorical('rf_max_depth', [80, 90, 100, 110])
        rf_max_features = trial.suggest_categorical('rf_max_features', [2, 3])
        rf_min_samples_leaf = trial.suggest_categorical('rf_min_sample_leaf', [8, 10, 12])
        rf_n_estimators = trial.suggest_categorical('rf_n_estimators', [100, 200, 300, 1000])
        
        model = RandomForestClassifier(
            random_state=SEED,
            max_depth=rf_max_depth,
            max_features=rf_max_features,
            min_samples_leaf=rf_min_samples_leaf,
            n_estimators=rf_n_estimators
        )

    elif classifier == 'LightGBM':
        # LightGBM params
        lgbm_max_depth = trial.suggest_int('lgbm_max_depth', 20, 200)
        lgbm_learning_rate = trial.suggest_categorical('lgbm_learning_rate', [0.01, 0.05, 0.1])
        lgbm_num_leaves = trial.suggest_categorical('lgbm_num_leaves', [80, 100, 150, 200])
#         lgbm_boosting_type = trial.suggest_categorical('lgbm_boosting_type', ['gbdt', 'dart', 'goss', 'rf'])
        lgbm_subsample = trial.suggest_categorical('lgbm_subsample', [1, 0.8, 0.7, 0.5])
        
        model = lgb.LGBMClassifier(
            random_state=SEED,
            max_depth=lgbm_max_depth,
            num_leaves=lgbm_num_leaves,
            learning_rate=lgbm_learning_rate,
            subsample=lgbm_subsample,
#             boosting_type=lgbm_boosting_type
        )

    elif classifier == 'Xgboost':
        # Xgboost params
        xgb_max_depth = trial.suggest_int('xgb_max_depth', 10, 200)
        xgb_learning_rate = trial.suggest_categorical('xgb_learning_rate', [0.01, 0.05, 0.1])
#         xgb_booster = trial.suggest_categorical('xgb_boosting_type', ['gbdt', 'dart', 'goss', 'rf'])
        
        model = xgb.XGBClassifier(
            random_state=SEED,
            max_depth=xgb_max_depth,
            learning_rate=xgb_learning_rate,
#             booster=xgb_booster
        )
        model
    else:
        return
        
    return cross_val_score(model, train_X, train_y, n_jobs=-1, scoring='accuracy', cv=kf).mean()

In [10]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=200)

trial = study.best_trial
print('Accuracy: {}'.format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

[I 2020-10-17 15:42:26,509] A new study created in memory with name: no-name-0782621e-9b6a-425c-8bfc-81b1457e77c4
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/roamgom/workspace/dankook_dacon/.venv/lib/python3.8/site-packages/joblib/parallel.py", line 921, in retrieve
    self._output.extend(job.get(timeout=self.timeout))
  File "/Users/roamgom/workspace/dankook_dacon/.venv/lib/python3.8/site-packages/joblib/_parallel_backends.py", line 542, in wrap_future_result
    return future.result(timeout=timeout)
  File "/usr/local/Cellar/python@3.8/3.8.5/Frameworks/Python.framework/Versions/3.8/lib/python3.8/concurrent/futures/_base.py", line 434, in result
    self._condition.wait(timeout)
  File "/usr/local/Cellar/python@3.8/3.8.5/Frameworks/Python.framework/Versions/3.8/lib/python3.8/threading.py", line 302, in wait
    waiter.acquire()
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/roamgom/workspace/dankook_dacon/.venv/lib/python3.8/site-packages/joblib/parallel.py", line 1042, in __call__
    self.retrieve(


KeyboardInterrupt

